In [1]:
%matplotlib inline
from keras.models import load_model
from keras.models import model_from_json
import numpy as np, librosa
from numpy import array
import glob
import os, shutil
from lxml import etree as ET
import datetime
import time

import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13, 5)

Using TensorFlow backend.


In [2]:
song_filepath = "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/file.mp3"
audio_segments_folder = "/Users/kcfdaniel/Downloads/fyp/audio_segments/"
tempo = 100
num_of_beats = 4
beat_subdivision = 4
pick_up_duration = 0
subdivision = 16
beat_duration = subdivision/beat_subdivision
bar_duration = num_of_beats*beat_duration

xml_fname = "/Users/kcfdaniel/Downloads/transcription.xml"
title = "transcription"
date = datetime.datetime.now().strftime("%Y-%m-%d")

In [3]:
if not song_filepath.split('/')[-1].split('.')[1] == "wav":
    y, sr = librosa.load(song_filepath, sr = 44100)
    song_filepath = '/'.join(song_filepath.split('/')[0:-1]) + '/temp.wav'
    librosa.output.write_wav(song_filepath, y, sr)

In [4]:
#load deep learning models
# transfered models
counter=0
start = time.time()
    
##load weight and structure seperately
models = []
names =["/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model1.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model2.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model3.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model5.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model6.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model8.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model10.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model11.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model15_binary_crossentropy.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model16_binary_crossentropy.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model17_binary_crossentropy_augment_dirty.h5",
        "/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/express react app/serverapp/deep_learning_models/final_model18_binary_crossentropy_augment_dirty.h5",]

for i in range(len(names)):
    name = names[i]
    json_name = name.split(".")[0]+".json"    
    json_file = open(json_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(name)
    models.append(model)
    counter += 1
    print(counter)
    
    
end = time.time()
print(end - start)

1
2
3
4
5
6
7
8
9
10
11
12
12.577983856201172


In [247]:
# models[0]
# test1 =  load_model("/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/myProject/Automatic_Drums_Transcription/deep_learning_models_integrated_test/final_model3.h5")
# test2 =  load_model("/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/myProject/Automatic_Drums_Transcription/deep_learning_models_integrated_test/final_model3.h5")
# models[0]
# load_model("/Users/kcfdaniel/Downloads/fyp/Automatic Drum Transcription/myProject/Automatic_Drums_Transcription/deep_learning_models_integrated_test/final_model3.h5")

In [248]:
# for i in range(len(models)):
#     model = models[i]
#     name = names[i]
#     json_name = name.split(".")[0]+".json"
#     model.save_weights(name)
#     model_json = model.to_json()
#     with open(json_name, "w") as json_file:
#         json_file.write(model_json)
#     json_file.close()

In [81]:
## FEATURE EXTRACTION function
progress_interval = (90 - progress) / len(filepaths)
progress = 78
counter = 0

sr = 44100
def extract_features(signal, features):
    #get CQT
    fvs = list()
    
    ## if stereo, change to mono
    if len(np.shape(signal)) == 2:
        signal = signal.T
        signal = librosa.to_mono(signal)
    
    result = librosa.cqt(signal, sr = sr, bins_per_octave = 12, n_bins=109)
    
    ######    slice to first n time slices   #####
    temp=[]
#     print(np.shape(result))
    for i in result:
        temp.append(i[:18])
    result = array(temp)
        
    
    fvs.append( result )
    
    global counter
    global progress
    counter += 1
    progress += progress_interval
    if counter == 20:
        print(str(int(progress)) + "%", flush=True)
        counter = 0
    
    return fvs

In [82]:
##extract percussive
y, sr = librosa.load(song_filepath, sr = sr)
y = np.hstack (((np.zeros(441)), y))
y_harmonic, y_percussive = librosa.effects.hpss(y)

# ipd.Audio(y,rate=sr)
##separate song according to onset
hop_length = 512
onset_frames = librosa.onset.onset_detect(y_percussive, sr=sr, hop_length=hop_length, backtrack=True)
onset_times = librosa.frames_to_time(onset_frames, sr=sr, hop_length=hop_length)
onset_samples = librosa.frames_to_samples(onset_frames, hop_length=hop_length)
segment_length = 8820

# print(len(onset_times))
# librosa.display.waveplot(y, sr)
# S = librosa.stft(y)
# logS = librosa.logamplitude(S)
# librosa.display.specshow(logS, sr=sr, x_axis='time', y_axis='log')
# plt.vlines(onset_times, 0, 10000, color='k')

KeyboardInterrupt: 

In [ ]:
print(len(onset_times))

In [ ]:
##clear contents in file storing audio segments
for the_file in os.listdir(audio_segments_folder):
    file_path = os.path.join(audio_segments_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
##write contents in file storing audio segments
for i in onset_samples:
    librosa.output.write_wav(audio_segments_folder + str(i) + ".wav", y_percussive[i: i+segment_length], sr)

In [83]:
##extract CQT out of presentation data
filepaths = glob.glob(audio_segments_folder+"*.wav")

temp = []
signals=[]
for p in filepaths:
    data, samplerate = librosa.load(p, sr = sr)
    if samplerate != sr:
        print(p)
        temp.append(samplerate)
    signals.append(data)

    
features = np.array([extract_features(x, 'cqt') for x in signals])
features = np.reshape(features, [np.shape(features)[0],np.shape(features)[2],np.shape(features)[3]])

#filepaths available
#features available

78%
78%
78%
78%
79%
79%
79%
79%
79%
80%
80%
80%
80%
81%
81%
81%
81%
81%
82%
82%
82%
82%
83%
83%
83%
83%
83%
84%
84%
84%
84%
85%
85%
85%
85%
85%
86%
86%
86%
86%
87%
87%
87%
87%
87%
88%
88%
88%
88%
89%
89%
89%
89%
89%


In [84]:
print(np.shape(features))
predictions = [model.predict(features) for model in models]  ##get predictions from models
predictions = np.array(predictions)  ##to array
print(np.shape(predictions))

predictions = np.delete(predictions,0,0)  ##remove the prediction from the 1st network
number_of_networks = np.shape(predictions)[0]
predictions = np.sum(predictions, axis = 0)  ##sum up all predictions across all networks
predictions = predictions/number_of_networks  ##divide by number of networks
predictions = predictions.tolist()  ##to list
##round to 0/1
temp = []
for i in predictions:
    temp.append([round(element, 0) for element in i])
predictions = temp
print(np.shape(predictions))


(1089, 109, 18)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


(12, 1089, 4)
(1089, 4)


In [300]:
## get drum_hit_envelope
filepaths_cleaned = np.array([[int(i.split(".wav")[0].split("/")[-1]) for i in filepaths]])  ##filepath/123.wav => 123   
print("np.shape(predictions): " + str(np.shape(predictions)))
print("len(filepaths_cleaned.T): " + str(len(filepaths_cleaned.T)))
predictions_filepaths = np.concatenate((predictions, filepaths_cleaned.T), axis=1)
predictions_filepaths = sorted(predictions_filepaths,key=lambda x: x[4])

seconds_per_minute = 60
filter_factor = 0.9
# offset = ( sr * seconds_per_minute / tempo / (subdivision/4) )*filter_factor
offset = 0
previous_sample = -offset
drum_hit_envelope = []

for i in predictions_filepaths:
    hit = False
#     print(i[4])
#     print(previous_frame + offset)
    if i[4] > (previous_sample + offset):
#         print("in!")
#         print()
        if i[0] == 1:
            drum_hit_envelope.append([str(i[4]), "kick"])
            hit = True
        if i[1] == 1:
            drum_hit_envelope.append([str(i[4]), "snare"])
            hit = True
        if i[2] == 1 and i[3] == 0:
            drum_hit_envelope.append([str(i[4]), "hhcl"])
            hit = True
        if i[3] == 1:
            drum_hit_envelope.append([str(i[4]), "hhop"])
            hit = True
    if hit:
        previous_sample = i[4]
        
    

np.shape(predictions): (17, 4)
len(filepaths_cleaned.T): 17


In [301]:
time_offset = float(drum_hit_envelope[0][0])
min_subdivision_duration = sr * seconds_per_minute / tempo / (subdivision/beat_subdivision)
## move all drum hit to start from time 0
drum_hit_envelope = [[float(i[0])-time_offset, i[1]] for i in drum_hit_envelope]
## convert time unit from sample to subdivision
drum_hit_envelope = [(round(i[0]/min_subdivision_duration) , i[1]) for i in drum_hit_envelope]
print(len(drum_hit_envelope))
drum_hit_envelope = sorted(list(set(drum_hit_envelope)))
print(len(drum_hit_envelope))
for drum_hit in drum_hit_envelope:
    print(drum_hit)

26
26
(0, 'hhcl')
(0, 'kick')
(2, 'hhcl')
(3, 'hhcl')
(3, 'snare')
(5, 'hhcl')
(7, 'hhcl')
(7, 'kick')
(8, 'hhcl')
(10, 'hhcl')
(10, 'snare')
(12, 'hhcl')
(13, 'hhcl')
(13, 'kick')
(15, 'hhcl')
(16, 'hhcl')
(16, 'snare')
(18, 'hhcl')
(20, 'hhcl')
(20, 'kick')
(22, 'hhcl')
(23, 'hhcl')
(23, 'snare')
(25, 'hhcl')
(26, 'hhcl')
(26, 'kick')


In [308]:
##contruct drums_xml

drums_xml = """"""
duration = 0

beat_count = 1
beat_duration
subdivision


current_drums = []
next_drum = None
debug = False
if pick_up_duration == 0:
    remaining_duration_in_bar = bar_duration
else:
    remaining_duration_in_bar = pick_up_duration
current_drums_duration = 0
first_note_in_chord = True
measure_number = 1

def get_subdivision(duration):
    ##return note name, # dots, any extra rest needed
    duration = duration * 16 / subdivision
    if duration == 1:
        return "16th", 0, ""
    elif duration == 2:
        return "eighth", 0, ""
    elif duration == 3:
        return "eighth", 1, ""
    elif duration == 4:
        return "quarter", 0, ""
    elif duration == 5:
        return "quarter", 0, note("rest", 1, False)
    elif duration == 6:
        return "quarter", 1, ""
    elif duration == 7:
        return "quarter", 2, ""
    elif duration == 8:
        return "quarter", 0, note("rest", 4, False)
    elif duration == 9:
        return "quarter", 0, note("rest", 5, False)
    elif duration == 10:
        return "quarter", 0, note("rest", 6, False)
    elif duration == 11:
        return "quarter", 0, note("rest", 7, False)
    elif duration == 12:
        return "quarter", 0, note("rest", 8, False)
    elif duration == 13:
        return "quarter", 0, note("rest", 9, False)
    elif duration == 14:
        return "quarter", 0, note("rest", 10, False)
    elif duration == 15:
        return "quarter", 0, note("rest", 11, False)
    elif duration == 16:
        return "quarter", 0, note("rest", 12, False)
    
def note(drum, duration, chord, drum_time):
#     print("2chord: "+ str(chord))
    print("9duration: "+ str(duration))
    print("9remaining_duration_in_bar: "+ str(remaining_duration_in_bar))
    print()
    
    t, dot, extra_rest = get_subdivision(duration)
    note_head_info = ""
    chord_info = ""
    dotted_info = ""
    for i in range(dot):
        dotted_info += """
        <dot/>"""
    
    if drum == "rest":
        return """
      <note>
        <rest/>
        <duration>"""+ str(duration) +"""</duration>
        <voice>1</voice>
        <type>"""+ t +"""</type>""" + dotted_info + """
      </note>"""
    if chord:
        chord_info = "<chord/>"
    if drum == "kick":
        attributes = ["F","4", str(duration), "P1-I37", t]
    if drum == "snare":
        attributes = ["C","5", str(duration), "P1-I39", t]
    if drum == "hhcl":
        attributes = ["G","5", str(duration), "P1-I43", t]
        note_head_info = "<notehead>x</notehead>"
    if drum == "hhop":
        attributes = ["G","5", str(duration), "P1-I47", t]
        note_head_info = "<notehead>circle-x</notehead>"
    return """
      <note>
      """+ chord_info +"""
        <unpitched>
          <display-step>"""+ attributes[0] +"""</display-step>
          <display-octave>"""+ attributes[1] +"""</display-octave>
          </unpitched>
        <duration>"""+ attributes[2] +"""</duration>
        <instrument id=\""""+ attributes[3] +"""\"/>
        <voice>1</voice>
        <type>"""+ attributes[4] +"""</type>""" + dotted_info + """
        <stem>up</stem>
        """+ note_head_info + """
      </note>""" + extra_rest

next_drum = drum_hit_envelope[0]
rest = False
for i in range(len(drum_hit_envelope)):
    current_drums.append(next_drum)
    if current_drums[0][0] == 720:
        debug = True
    else:
        debug = False
    if debug == True:
        for drum in current_drums:
            print(drum)
#     print("current drum time: "+ str(current_drums[0][0]))
#     print("i: " + str(i))
#     print("len(drum_hit_envelope)-1: " + str(len(drum_hit_envelope)-1))
    ##if last note
    if (i == len(drum_hit_envelope)-1):
        print("last note!")
        ##if chord
        first_note_in_chord = True
        duration = bar_duration - current_drums[0][0]%bar_duration
        print("current_drums[0][0]: "+str(current_drums[0][0]))
        print("bar_duration: "+str(bar_duration))
        print("duration: "+str(duration))
        if duration <= beat_duration:
            print("last short note!")
            current_drums_duration = duration
            for drum in current_drums:
                print(drum)
                print("current_drums_duration: " + str(current_drums_duration))
                drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                first_note_in_chord = False
            current_drums = []
        else:
            print("last long note!")
            if remaining_duration_in_bar%beat_duration == 0:
                current_drums_duration = beat_duration
            else:
                current_drums_duration = remaining_duration_in_bar%beat_duration
            duration -= current_drums_duration
            for drum in current_drums:
                print(drum)
                print("current_drums_duration: " + str(current_drums_duration))
                drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                first_note_in_chord = False
            current_drums = []
            ##append rest
            while duration > beat_duration:
                duration = duration - beat_duration
                drums_xml += note("rest", beat_duration, False, "")
            if duration > 0:
                drums_xml += note("rest", duration, False, "")
    else:
        next_drum = drum_hit_envelope[i+1]
        ##if can write drums now
        if current_drums[0][0] < next_drum[0]:
            print("current drum time: "+ str(current_drums[0][0]))
            print("next drum time: "+ str(next_drum[0]))
            duration = next_drum[0] - current_drums[0][0]
            ##write normal drums
            if duration <= beat_duration and duration < remaining_duration_in_bar:
                if remaining_duration_in_bar%beat_duration == 0:
                    current_drums_duration = duration
                else:
                    print("2.0 here remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                    current_drums_duration = min(remaining_duration_in_bar%beat_duration, duration)
                    rest_duration = duration - current_drums_duration
                    print("2.0 here duration: " + str(duration))
                    print("2.0 here current_drums_duration: " + str(current_drums_duration))
                    print("2.0 here rest_duration: " + str(rest_duration))
                    if rest_duration != 0:
                        rest = True
                print("2.0remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                first_note_in_chord = True
                for drum in current_drums:
                    print(drum)
                    print("2.0current_drums_duration: " + str(current_drums_duration))
                    print("2.0first_note_in_chord: "+str(first_note_in_chord))
                    drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                    first_note_in_chord = False
                current_drums = []
                remaining_duration_in_bar -= current_drums_duration
                if rest:
                    print("2.1")
                    drums_xml += note("rest", rest_duration, False, "")
                    remaining_duration_in_bar -= rest_duration
                    rest = False
            ##write drums with next rest at the beginning of next bar
            elif duration <= beat_duration and duration > remaining_duration_in_bar:
                current_drums_duration = remaining_duration_in_bar
                duration = duration - current_drums_duration
                first_note_in_chord = True
                for drum in current_drums:
                    print(drum)
                    print("3remaining_duration_in_bar: "+ str(remaining_duration_in_bar))
                    print("3current_drums_duration: " + str(current_drums_duration))
                    drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                    first_note_in_chord = False
                current_drums = []
                remaining_duration_in_bar = bar_duration
                ##start next bar
                measure_number += 1
                drums_xml += """
                </measure>
        <measure number=\""""+ str(measure_number) +"""\">"""
                ##write rest at beginning of next bar
                print("3remaining_duration_in_bar: "+ str(remaining_duration_in_bar))
                drums_xml += note("rest", duration, False, "")
                remaining_duration_in_bar -= duration 
            ##write drums with next note at the beginning of next bar
            elif duration <= beat_duration and duration == remaining_duration_in_bar:
                current_drums_duration = duration
                remaining_duration_in_bar = bar_duration
                first_note_in_chord = True
                for drum in current_drums:
                    print(drum)
                    print("4current_drums_duration: " + str(current_drums_duration))
                    drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                    first_note_in_chord = False
                current_drums = []
                ##start next bar
                measure_number += 1
                drums_xml += """
                </measure>
        <measure number=\""""+ str(measure_number) +"""\">"""
            ##write long drums before or until next bar
            elif duration > beat_duration and duration <= remaining_duration_in_bar: 
                print("5.0remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                print("5.0duration: " + str(duration))
                until_next_bar = (duration == remaining_duration_in_bar)
                if remaining_duration_in_bar%beat_duration == 0:
                    current_drums_duration = beat_duration
                else:
                    print("5.2 here remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                    current_drums_duration = remaining_duration_in_bar%beat_duration
                duration -= current_drums_duration
                first_note_in_chord = True
                for drum in current_drums:
#                     print("current_drums: ")
#                     for d in current_drums:
#                         print(d)
#                     print()
                    print(drum)
                    print("5.1remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                    print("5current_drums_duration: " + str(current_drums_duration))
                    drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                    first_note_in_chord = False
                current_drums = []
                remaining_duration_in_bar -= current_drums_duration
                ##append rest
                while duration > beat_duration:
                    duration = duration - beat_duration
                    print("5.1remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                    drums_xml += note("rest", beat_duration, False, "")
                    remaining_duration_in_bar -= beat_duration
                print("5.1remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                if duration > 0:
                    print("5.3append rest: " + str(duration))
                    drums_xml += note("rest", duration, False, "")
                remaining_duration_in_bar -= duration
                print("5.3remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                if until_next_bar:
                    ##start next bar
                    print("here")
                    remaining_duration_in_bar = bar_duration
                    measure_number += 1
                    drums_xml += """
                    </measure>
            <measure number=\""""+ str(measure_number) +"""\">"""
                print("5remaining_duration_in_bar: "+ str(remaining_duration_in_bar))
            ##write long drums with next drum after first beat of next bar
            elif duration > beat_duration and duration > remaining_duration_in_bar:
                print("6remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                print("6duration: " + str(duration))
                if remaining_duration_in_bar%beat_duration == 0:
                    current_drums_duration = beat_duration
                else:
                    print("6.2 here remaining_duration_in_bar: " + str(remaining_duration_in_bar))
                    current_drums_duration = remaining_duration_in_bar%beat_duration
                duration -= current_drums_duration
                first_note_in_chord = True
                for drum in current_drums:
                    print(drum)
                    print("6current_drums_duration: " + str(current_drums_duration))                    
                    drums_xml += note(drum[1], current_drums_duration, not first_note_in_chord, drum[0])
                    first_note_in_chord = False
                current_drums = []
                remaining_duration_in_bar -= current_drums_duration
                if remaining_duration_in_bar == 0:
                    remaining_duration_in_bar = bar_duration
                    current_drums = []
                    ##start next bar
                    measure_number += 1
                    drums_xml += """
                    </measure>
            <measure number=\""""+ str(measure_number) +"""\">"""
                    ##write rest at beginning of next bar
                    ##append rest
                while duration > beat_duration:
                    duration = duration - beat_duration
                    print("6.1write rest with duration: " + str(beat_duration))   
                    drums_xml += note("rest", beat_duration, False, "")
                    remaining_duration_in_bar -= beat_duration
#                     print("7remaining_duration_in_bar: " + str(remaining_duration_in_bar))   
                    if remaining_duration_in_bar == 0 :
                        ##start next bar
                        remaining_duration_in_bar = bar_duration
                        measure_number += 1
                        drums_xml += """
                        </measure>
                <measure number=\""""+ str(measure_number) +"""\">"""
                print("8duration: " + str(duration))
                drums_xml += note("rest", duration, False, "")
                remaining_duration_in_bar -= duration
                if remaining_duration_in_bar == 0 :
                        ##start next bar
                        remaining_duration_in_bar = bar_duration
                        measure_number += 1
                        drums_xml += """
                        </measure>
                <measure number=\""""+ str(measure_number) +"""\">"""

                
current_drums.append(drum_hit_envelope[-1])

current drum time: 0
next drum time: 2
2.0remaining_duration_in_bar: 16.0
(0, 'hhcl')
2.0current_drums_duration: 2
2.0first_note_in_chord: True
9duration: 2
9remaining_duration_in_bar: 16.0

(0, 'kick')
2.0current_drums_duration: 2
2.0first_note_in_chord: False
9duration: 2
9remaining_duration_in_bar: 16.0

current drum time: 2
next drum time: 3
2.0 here remaining_duration_in_bar: 14.0
2.0 here duration: 1
2.0 here current_drums_duration: 1
2.0 here rest_duration: 0
2.0remaining_duration_in_bar: 14.0
(2, 'hhcl')
2.0current_drums_duration: 1
2.0first_note_in_chord: True
9duration: 1
9remaining_duration_in_bar: 14.0

current drum time: 3
next drum time: 5
2.0 here remaining_duration_in_bar: 13.0
2.0 here duration: 2
2.0 here current_drums_duration: 1.0
2.0 here rest_duration: 1.0
2.0remaining_duration_in_bar: 13.0
(3, 'hhcl')
2.0current_drums_duration: 1.0
2.0first_note_in_chord: True
9duration: 1.0
9remaining_duration_in_bar: 13.0

(3, 'snare')
2.0current_drums_duration: 1.0
2.0first_no

In [309]:
# print(drums_xml)

In [310]:
tempo_beat_unit = ""
if beat_subdivision == 2:
    tempo_beat_unit = "half"
if beat_subdivision == 4:
    tempo_beat_unit = "quarter"
elif beat_subdivision == 8:
    tempo_beat_unit = "eighth"

doc = ET.fromstring("""<score-partwise>
  <work>
    <work-title>"""+ title +"""</work-title>
  </work>
  <identification>
    <encoding>
      <software>MuseScore 2.1.0</software>
      <encoding-date>"""+ date +"""</encoding-date>
      <supports element="accidental" type="yes"/>
      <supports element="beam" type="yes"/>
      <supports element="print" attribute="new-page" type="yes" value="yes"/>
      <supports element="print" attribute="new-system" type="yes" value="yes"/>
      <supports element="stem" type="yes"/>
    </encoding>
  </identification>
  <defaults>
    <scaling>
      <millimeters>7.05556</millimeters>
      <tenths>40</tenths>
    </scaling>
    <page-layout>
      <page-height>1683.36</page-height>
      <page-width>1190.88</page-width>
      <page-margins type="even">
        <left-margin>56.6929</left-margin>
        <right-margin>56.6929</right-margin>
        <top-margin>56.6929</top-margin>
        <bottom-margin>113.386</bottom-margin>
      </page-margins>
      <page-margins type="odd">
        <left-margin>56.6929</left-margin>
        <right-margin>56.6929</right-margin>
        <top-margin>56.6929</top-margin>
        <bottom-margin>113.386</bottom-margin>
      </page-margins>
    </page-layout>
    <word-font font-family="FreeSerif" font-size="10"/>
    <lyric-font font-family="FreeSerif" font-size="11"/>
  </defaults>
  <credit page="1">
    <credit-words default-x="595.44" default-y="1626.67" justify="center" valign="top" font-size="24">"""+ title +"""</credit-words>
  </credit>
  <part-list>
    <score-part id="P1">
      <part-name>Drumset</part-name>
      <part-abbreviation>Drs.</part-abbreviation>
      <score-instrument id="P1-I36">
        <instrument-name>Acoustic Bass Drum</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I37">
        <instrument-name>Bass Drum 1</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I38">
        <instrument-name>Cross Stick</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I39">
        <instrument-name>Acoustic Snare</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I41">
        <instrument-name>Electric Snare</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I42">
        <instrument-name>Low Floor Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I43">
        <instrument-name>Closed Hi-Hat</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I44">
        <instrument-name>High Floor Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I45">
        <instrument-name>Pedal Hi-Hat</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I46">
        <instrument-name>Low Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I47">
        <instrument-name>Open Hi-Hat</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I48">
        <instrument-name>Low-Mid Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I49">
        <instrument-name>Hi-Mid Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I50">
        <instrument-name>Crash Cymbal 1</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I51">
        <instrument-name>High Tom</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I52">
        <instrument-name>Ride Cymbal 1</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I53">
        <instrument-name>Chinese Cymbal</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I54">
        <instrument-name>Ride Bell</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I55">
        <instrument-name>Tambourine</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I56">
        <instrument-name>Splash Cymbal</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I57">
        <instrument-name>Cowbell</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I58">
        <instrument-name>Crash Cymbal 2</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I59">
        <instrument-name>Fill</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I60">
        <instrument-name>Ride Cymbal 2</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I64">
        <instrument-name>Open Hi Conga</instrument-name>
        </score-instrument>
      <score-instrument id="P1-I65">
        <instrument-name>Low Conga</instrument-name>
        </score-instrument>
      <midi-device port="1"></midi-device>
      <midi-instrument id="P1-I36">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>36</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I37">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>37</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I38">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>38</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I39">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>39</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I41">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>41</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I42">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>42</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I43">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>43</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I44">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>44</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I45">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>45</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I46">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>46</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I47">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>47</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I48">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>48</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I49">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>49</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I50">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>50</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I51">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>51</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I52">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>52</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I53">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>53</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I54">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>54</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I55">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>55</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I56">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>56</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I57">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>57</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I58">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>58</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I59">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>59</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I60">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>60</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I64">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>64</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
        </midi-instrument>
      <midi-instrument id="P1-I65">
        <midi-channel>10</midi-channel>
        <midi-program>1</midi-program>
        <midi-unpitched>65</midi-unpitched>
        <volume>78.7402</volume>
        <pan>0</pan>
      </midi-instrument>
    </score-part>
  </part-list>
  <part id="P1">
    <measure number="1" width="476.40">
      <print>
        <system-layout>
          <system-margins>
            <left-margin>0.00</left-margin>
            <right-margin>601.09</right-margin>
            </system-margins>
          <top-system-distance>170.00</top-system-distance>
          </system-layout>
        </print>
      <attributes>
        <divisions>""" + str(beat_duration) + """</divisions>
        <key>
          <fifths>0</fifths>
          </key>
        <time>
          <beats>"""+str(num_of_beats)+"""</beats>
          <beat-type>"""+str(beat_subdivision)+"""</beat-type>
          </time>
        <clef>
          <sign>percussion</sign>
          <line>2</line>
          </clef>
        </attributes>
      <direction placement="above">
        <direction-type>
          <metronome parentheses="no">
            <beat-unit>"""+tempo_beat_unit+"""</beat-unit>
            <per-minute>"""+ str(tempo) + """</per-minute>
            </metronome>
          </direction-type>
        <sound tempo=\"""" + str(tempo*(4/beat_subdivision)) + """\"/>
        </direction>""" + 
drums_xml+
"""
      <barline location="right">
        <bar-style>light-heavy</bar-style>
      </barline>
    </measure>
  </part>
</score-partwise>
""")

tree = doc.getroottree()

# print(ET.tostring(tree, encoding='unicode', pretty_print=True))
# print(tree)
tree.write(xml_fname, pretty_print=True, encoding='utf-8')

In [72]:
y = 1
def add():
    global y
    y += 1
    return y

In [77]:
add()

5